In [1]:
!pip install python-terrier
!pip install lxml
import numpy as np
import sys

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import pyterrier as pt
if not pt.started():
  pt.init()

from bs4 import BeautifulSoup as bsoup
import pyterrier.io
from pyterrier.measures import *
import html
import os, glob
import argparse
from tqdm import tqdm

     |████████████████████████████████| 97 kB 2.8 MB/s 
     |████████████████████████████████| 1.1 MB 12.1 MB/s 
     |████████████████████████████████| 69 kB 3.8 MB/s 
     |████████████████████████████████| 297 kB 37.0 MB/s 
     |████████████████████████████████| 46 kB 1.9 MB/s 
     |████████████████████████████████| 45 kB 2.4 MB/s 
     |████████████████████████████████| 294 kB 36.3 MB/s 
     |████████████████████████████████| 596 kB 14.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 126 kB 47.9 MB/s 
     |████████████████████████████████| 6.4 MB 57.7 MB/s 
     |████████████████████████████████| 1.2 MB 43.8 MB/s 
     |████████████████████████████████| 72 kB 552 kB/s 
     |████████████████████████████████| 287 kB 47.5 MB/s 
     |████████████████████████████████| 59 kB 6.7 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.1-py3-none

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
# !wget https://www.cs.rit.edu/~dprl/data/ARQMath/ARQMath_Collection.zip
# !unzip ARQMath_Collection.zip
# !rm ARQMath_Collection.zip

In [2]:
# Get the tokenized and stemmed/stopped post index
!wget https://www.cs.rit.edu/~dprl/data/ARQMath/ARQMath_Collection-post-ptindex.zip
!unzip ARQMath_Collection-post-ptindex.zip
!rm ARQMath_Collection-post-ptindex.zip

#Get the tokenized math index
!wget https://www.cs.rit.edu/~dprl/data/ARQMath/ARQMath_Collection-math-ptindex.zip
!unzip ARQMath_Collection-math-ptindex.zip
!rm ARQMath_Collection-math-ptindex.zip

--2022-04-28 02:03:38--  https://www.cs.rit.edu/~dprl/data/ARQMath/ARQMath_Collection-post-ptindex.zip
Resolving www.cs.rit.edu (www.cs.rit.edu)... 129.21.30.104
Connecting to www.cs.rit.edu (www.cs.rit.edu)|129.21.30.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1674284873 (1.6G) [application/zip]
Saving to: ‘ARQMath_Collection-post-ptindex.zip’

ARQMath_Collection- 100%[===================>]   1.56G   106MB/s    in 17s     

2022-04-28 02:03:55 (96.1 MB/s) - ‘ARQMath_Collection-post-ptindex.zip’ saved [1674284873/1674284873]

Archive:  ARQMath_Collection-post-ptindex.zip
   creating: ARQMath_Collection-post-ptindex/
  inflating: ARQMath_Collection-post-ptindex/data.lexicon.fsomapid  
  inflating: ARQMath_Collection-post-ptindex/data.inverted.bf  
  inflating: ARQMath_Collection-post-ptindex/data.direct.bf  
  inflating: ARQMath_Collection-post-ptindex/data.lexicon.fsomaphash  
  inflating: ARQMath_Collection-post-ptindex/data.lexicon.fsomapfile  
  inf

In [4]:
#Mounting Colab Notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cd drive/MyDrive/ARQMath_Project

In [7]:
def load_qrels( file_name ):
    qrels = pt.io.read_qrels( file_name )
    return qrels
    
load_qrels('qrel_official_task1')

,qid,docno,label
0,A.1,2190525,0
1,A.1,2761594,2
2,A.1,2657317,0
3,A.1,109097,0
4,A.1,2465924,0
...,...,...,...
39119,A.99,3004212,0
39120,A.99,3013821,0
39121,A.99,3015907,0
39122,A.99,3036216,0


In [8]:
################################################################
# math_recoding.py
#
# Dictionaries and functions to recode formula encodings (e.g.,
# LaTeX) for use with PyTerrier, which removes punctuation by
# default.
#
# Author:
# R. Zanibbi, April 2022 (CSCI 539, Information Retrieval, RIT)
################################################################

################################################################
# LaTeX Symbol Map
################################################################
# Warning - mapping may be incomplete 
# Commands will be tokenized as-is 
# (e.g., \geq -> backslash ge )
#
# Modification:
#
# Also mapping punctuation to full words, in hopes of better matching
# tokenization in existing BERT models, and to capture more coreferences
# when text and formulas refer to the same thing/concept (i.e., increasing
# the likelihood of the 'same name for the same thing').
latex_symbol_map = {
        "+" : " plus ",
        "-" : " minus ",
        "^" : " power ",
        "*" : " star ",

        "#" : " hash ",
        "!" : " exclaim ",
        "?" : " question ", 
        "," : " comma ",
        "." : " period ",
        "/" : " forward slash ",
        ":" : " colon ",
        ";" : " semicolon ",
        "&" : " ampersand ",
        "~" : " tilde ",
        "`" : " backquote ",
        "@" : " at ",
        "$" : " dollar ",
        "%" : " percent ",
        '"' : " double quote ",
        "'" : " single quote ",
        "|" : " vertical bar ",

        "=" : " equals ",
        ">" : " greater than ",
        "<" : " less than ",

        "{" : " open brace ",
        "}" : " close brace ",
        "[" : " open square bracket ",
        "]" : " close square bracket ",
        "(" : " open parenthesis ",
        ")" : " close parenthesis ",
        
        "\\\\" : " double backslash ",
        "\\" : " backslash "
    }

# 'Mini' query language for formulas, replacing pyterrier ops        
# e.g., using '_ptand' for the AND (+) operator
# e.g., using '_ptnot' for the NOT (-) operator
pyterrier_symbol_map = {
        "_pand":       "+",
        "_pnot":       "-",
        "_pobrace":    "{",
        "_pcbrace":    "}"
    }


################################################################
# Functions to transform strings encoding math
################################################################
def rewrite_symbols( in_string, map_dict ):
    out_string = in_string

    # Brute force - apply rules one-at-a-time
    for key in map_dict:
        out_string = out_string.replace( key, map_dict[ key ] )

    return out_string

def translate_latex( TeXstring ):
    # Replace LaTeX symbols in a string by text tokens
    return rewrite_symbols( TeXstring, latex_symbol_map )

def translate_query( query):
    # Translate query string to 'arqmath pyterrier query language' representation
    # (translate TeX symbols + 'meta' operators (e.g., _pand for '+' in pyterrier query language)
    return rewrite_symbols(
            rewrite_symbols( query, latex_symbol_map ),
            pyterrier_symbol_map 
        )

def translate_qlist( query_list ):
    # For batch retrieval
    return list( map( translate_query, query_list ) )


In [23]:

################################################################
# index_arqmath.py
#
# PyTerrier-based Python program for indexing ARQMath data
#
# Author:
# R. Zanibbi, April 2022 (CSCI 539, Information Retrieval, RIT)
################################################################

# Constants for indexing
# **Warning: tag names converted to lower case by default in BSoup (e.g., <P> -> <p>)
TAGS_TO_REMOVE = ['p','a','body','html','question','head','title']

TEXT_RETRIEVAL_FIELDS = [ 'title', 'text', 'tags', 'parentno' ]
TEXT_META_FIELDS = ['docno','title', 'text', 'origtext', 'tags', 'votes', 'parentno', 'mathnos' ]
TEXT_META_SIZES = [ 16, 256, 4096, 4096, 128, 8, 20, 20 ]

MATH_RETRIEVAL_FIELDS = [ 'text', 'parentno' ]
MATH_META_FIELDS = [ 'docno', 'text', 'origtext','postno','parentno']
MATH_META_SIZES = [ 20, 1024, 1024, 20, 20]

EMPTY_DOCS = 0

################################################################
# Index creation and properties
################################################################
def remove_tags( soup_node, tag_list ):
    for tag in soup_node( tag_list ):
        tag.unwrap()

def rewrite_math_tags( soup ):
    # Skip span tags without id's (i.e., formulas without identifiers)
    # Skip empty regions.
    formulaTags = [ node for node in soup('span') if node.has_attr('id') ]
    formula_ids = [ node['id'] for node in formulaTags if node.has_attr('id') ]

    #dbshow( "soup", soup )
    #dbshow( "soup('span')", soup('span') )
    #dbshow( "formulaTags", formulaTags )
    #pshow( "formula_ids", formula_ids )

    for tag in formulaTags:
        tag.name = 'math'
        del tag['class']
        #del tag['id']

    return ( formulaTags, formula_ids )

def print_formula_record(math_tag, tokenized_formula, docno, parentno ):
    print('\nDOCNO (FORMULA ID):',math_tag['id'],'\nTEXT:',tokenized_formula,'\nORIGTEXT:',math_tag.get_text(),'\nPOSTNO:',docno,
                                    '\nPARENTNO',parentno)

def print_post_record( docno, title_text, modified_post_text, indexed_body,
        tag_text, all_formula_ids, parentno, votes):
    print('\nDOCNO: ',docno,'\nTITLE: ',title_text,'\nBODY: ',modified_post_text,
        '\nTEXT (SEARCHABLE):',indexed_body,'\nTAGS: ',tag_text,'\nMATHNOS:',all_formula_ids,
        '\nPARENTNO:',parentno,'\nVOTES:',votes)

def generate_XML_post_docs(file_name_list, formula_index=False, debug_out=False ):
    global EMPTY_DOCS

    for file_name in file_name_list:
        print(">> Reading File: ", file_name )
        with open(file_name) as xml_file:
            soup = bsoup(xml_file, 'lxml')
            rows = soup('row')

#            for row in tqdm( rows ): 
            for row in tqdm( rows[0:1000] ): #reads only the first 1000 documents, just for demo speed
                # Parse post body and title content as HTML & get formulas
                # Document number in collection, user votes
                docno = row['id']
                votes = row['score']

                # Parent post for answers ('qpost' for questions)
                parentno = 'qpost'
                if row[ 'posttypeid' ] == '2':
                    parentno = row['parentid']

                # Title formulas - apply soup to recover HTML structure from attribute field value
                title_soup = bsoup( html.unescape( row.get('title','') ), 'lxml' )
                remove_tags( title_soup, TAGS_TO_REMOVE )
                ( title_formulas, title_formula_ids ) = rewrite_math_tags( title_soup )

                # Body formulas and simplification - again, apply soup to construct Tag tree w. bsoup
                body_soup = bsoup( html.unescape( row.get('body','') ), 'lxml' )
                remove_tags( body_soup, TAGS_TO_REMOVE )
                ( body_formulas, formula_ids )= rewrite_math_tags( body_soup )

                # Remove tags that we do not want to search.

                # Combine title and body formulas
                all_formulas = title_formulas + body_formulas
                all_formula_ids = title_formula_ids + formula_ids

                if formula_index:
                    ## Formula index entries
                    #  One output per formula
                    for math_tag in all_formulas:
                        raw_text = math_tag.get_text()
                        tokenized_formula = rewrite_symbols( math_tag.get_text(), latex_symbol_map )

                        # Skip empty formulas
                        if tokenized_formula.isspace():
                            EMPTY_DOCS += 1
                            if debug_out:
                                if raw_text.isspace():
                                    print('!!! WARNING: Empty "text" field for retrieval, formula id: ' + math_tag['id'] )
                                else:
                                    print('!!! WARNING: Non-empty formula tokenized into empty string, formula id: ' + math_tag['id'])
                                ##print_formula_record(math_tag, tokenized_formula, docno, parentno )
                            continue

                        elif debug_out:
                            print_formula_record( math_tag, tokenized_formula, docno, parentno )

                        yield { 'mathno':     math_tag['id'],
                                'text':      tokenized_formula,
                                'origtext':  math_tag.get_text(),
                                'docno':    docno,
                                'parentno' : parentno
                            }
                else:
                    ## Post text index entries ##
                    # Remove formula ids from title and body
                    #for math_tag in all_formulas:
                    #    del math_tag['id']

                    # Generate strings for title, post body, and tags
                    title_text = str( title_soup )
                    modified_post_text = str( body_soup )
                    indexed_body = translate_latex( modified_post_text )
                    tag_text = row.get('tags', '').replace('<','').replace('>',', ').replace('-',' ')

                    # Skip posts with empty content
                    if indexed_body.isspace():
                        EMPTY_DOCS += 1
                        if debug_out:
                            print('!!! WARNING: Empty "text" field for retrieval, post id: ' + docno )
                            #print_post_record( docno, title_text, modified_post_text, indexed_body,
                            #    tag_text, all_formula_ids, parentno, votes)
                        continue

                    elif debug_out:
                        print_post_record( docno, title_text, modified_post_text, indexed_body,
                            tag_text, all_formula_ids, parentno, votes)

                    # Note: the formula ids are stored in a string currently.
                    # Concatenate post and tag text
                    # NOTE: representation for search is tokenized differently than meta/document index version for viewing hits
                    yield { 'docno' :   docno,
                            'title' :   title_text,
                            'text' :    indexed_body,
                            'origtext': modified_post_text,
                            'tags' :    tag_text,
                            'mathnos' : all_formula_ids,
                            'parentno': parentno,
                            'votes' :   votes
                        }


def create_XML_index( file_list, indexName, token_pipeline="Stopwords,PorterStemmer", formulas=False, debug=False):
    # Storing processed text AND original text in meta index, docs, to support neural reranking with keywords, and
    # viewing original posts
    ( meta_fields, meta_sizes ) = ( TEXT_META_FIELDS, TEXT_META_SIZES )
    field_names= TEXT_RETRIEVAL_FIELDS

    if formulas:
        ( meta_fields, meta_sizes ) = ( MATH_META_FIELDS, MATH_META_SIZES )
        field_names= MATH_RETRIEVAL_FIELDS

    indexer = pt.IterDictIndexer(
            indexName,
            meta=meta_fields,
            meta_lengths=meta_sizes,
            overwrite=True )

    indexer.setProperty( "termpipelines", token_pipeline )
    index_ref = indexer.index( generate_XML_post_docs( file_list, formula_index=formulas, debug_out=debug ), fields=field_names )

    if EMPTY_DOCS > 0:
        count = str( EMPTY_DOCS )
        print("*** WARNING: " + count + " documents/formulas empty before tokenization, and were skipped.")
        print("    Additional documents/formulas may be empty after tokenization (PyTerrier message will report)")

    return pt.IndexFactory.of( index_ref )

## Visualization routines

def show_tokens( index ):
    # Show lexicon entries
    for kv in index.getLexicon():
        print("%s :    %s" % (kv.getKey(), kv.getValue().toString()) )

def show_index_stats( index ):
    print( index.getCollectionStatistics().toString() )

def view_index( indexName, index, view_tokens, view_stats ):
    if view_tokens or view_stats:
        print('\n[ ' + indexName + ': Details ]')
        if view_stats:
            show_index_stats( index )
        if view_tokens:
            print('Lexicon for ' + indexName + ':')
            show_tokens( index )
            print('')


In [10]:
post_index = '/content/ARQMath_Collection-post-ptindex'
math_index = '/content/ARQMath_Collection-math-ptindex'

In [15]:
def replace_formulas( soup_tag ):
    # Skip span tags without id's (i.e., formulas without identifiers)
    formulaTags = soup_tag('span')
    for tag in formulaTags:
        tag.name = 'math'
        del tag['class']
        del tag['id']

def convert_topic( topic_tag ):
    # Topic number and tags
    topic_number = topic_tag['number']
    tags = ''
    if topic_tag.tags:
        tags = html.unescape( topic_tag.tags.get_text() )   

    # Convert fields to index representation
    # HACK: Removing dollar signs for formulas (appear absent in the index)
    # DEBUG: Beautiful soup 'get_text()' removes all tags; use str for full trees
    title_text =  html.unescape( str( topic_tag('title')[0] )).replace('$','')
    body_text =   html.unescape( str( topic_tag('question')[0])).replace('$','')

    title_soup = bsoup( title_text,  'lxml')
    body_soup = bsoup( body_text, 'lxml')

    replace_formulas( title_soup )
    replace_formulas( body_soup )

    # Remove pruned tags (defined in index_arqmath.py)
    remove_tags( title_soup, TAGS_TO_REMOVE )
    remove_tags( body_soup, TAGS_TO_REMOVE )

    # REWRITE punctuation/math before exporting
    # NOTE: To see original query, please consult original topics files

    title_text = rewrite_symbols( title_soup.get_text(), latex_symbol_map )
    body_text =  rewrite_symbols(body_soup.get_text(), latex_symbol_map )
    
    # Save full query as TITLE field only (first version)
    # **IMPORTANT**: '- qpost' prepended so that Terrier returns only answer posts.
    #   This can be removed if you want to see which questions match as well.
    topic_text = '-qpost ' + title_text 

    # BUG: the output text does not contain the 'math' tokens for math tags present
    # in the indexed documents. Skipping for time (BM25 is a bag-of-words model).
    return ( topic_number, topic_text, title_text, body_text, tags )

def read_topic_file( file_name ):
    with open( file_name ) as infile:
        topic_soup = bsoup( infile, 'lxml' )
        topic_tags = topic_soup('topic')

        # Extract fields
        tuples = [ convert_topic(tag) for tag in topic_tags ]
        print(tuples)
        df = pd.DataFrame( tuples, columns= [  'qid', 'query','title','body','tags' ] )
        #filtered df to pass as query_list
        df2 = df[['qid','query']]

        return ( len( topic_tags ), df2 )

In [16]:
read_topics = read_topic_file('/content/Topics_V2.0.xml')

[('A.1', '-qpost Finding value of c such that the range of the rational function f open parenthesis x close parenthesis   equals   backslash frac open brace x power 2  plus  x  plus  c close brace  open brace x power 2  plus  2x  plus  c close brace  does not contain  open square bracket  minus 1 comma   minus  backslash frac open brace 1 close brace  open brace 3 close brace  close square bracket ', 'Finding value of c such that the range of the rational function f open parenthesis x close parenthesis   equals   backslash frac open brace x power 2  plus  x  plus  c close brace  open brace x power 2  plus  2x  plus  c close brace  does not contain  open square bracket  minus 1 comma   minus  backslash frac open brace 1 close brace  open brace 3 close brace  close square bracket ', 'I am comfortable when I am asked to calculate the range of a rational function comma  but how do we do the reverse question  I came across this problem period  If f open parenthesis x close parenthesis  equa

In [20]:
################################################################
# Search engine construction and search
################################################################
def search_engine( index,
        model,
        metadata_keys=[],
        token_pipeline="Stopwords,PorterStemmer" ):
    return pt.BatchRetrieve( index, wmodel=model,
            properties={ "termpipelines" : token_pipeline },
            metadata = metadata_keys )

# Run a single query
def query( engine, query ):
    return engine.search( translate_query( query ) )

# Run a list of queries
def batch_query( engine, query_list ):
    column_names=[ "qid", "query" ]

    query_count = len(query_list)
    qid_list = [ str(x) for x in range(1, query_count + 1) ]

    # Map TeX characters and ARQMath-version query ops (e.g., '_pand' -> '+')
    print( query_list )
    rewritten_query_list = translate_qlist( query_list )

    query_pairs = list( zip( qid_list, rewritten_query_list ) )
    queries = pd.DataFrame( query_pairs, columns=column_names )

    return engine( queries )

def verbose_hit_summary( result, math_index=False ):

    result.reset_index()
    for ( index, row ) in result.iterrows():
        #print("KEYS: " + str( row.keys() ) )
        print('QUERY (' + row['qid'] + '): ', row['query'])
        score = "{:.2f}".format( row['score'] )

        print('RANK:', index, 'Score:', score)
        if not math_index:
            # Post document
            print('Docid:',row['docid'], 'Post-no:', row['docno'], 'Parent-no:',row['parentno'],'Votes:', row['votes'] )
            if row['parentno'] == 'qpost':
                print('QUESTION TITLE:', row['title'])
            else:
                print('ANSWER')
        else:
            # Formula document
            print('Docid:',row['docid'], 'Formula-no:', row['docno'],  'Post-no (docno):', row['docno'], 'Parent-no:',row['parentno'])

        # Show original text before token mapping
        print('TEXT:\n', row['text'])
        print('ORIGTEXT:\n',row['origtext'])

        # Provide tags, formula id's for posts
        if not math_index:
            print('TAGS:',row['tags'])
            print('FORMULA IDS:',row['mathnos'])

        print('')

def show_result( result, field_names=[], show_table=True, show_hits=False, math=False ):
    print("\n__ SEARCH RESULTS _________________________\n")

    if show_table:
        if field_names == []:
            print( result, '\n' )
        else:
            print( result[ field_names ], '\n' )

    if show_hits:
        verbose_hit_summary( result, math_index=math )

def test_retrieval( k, post_index, math_index, model, tokens, debug=False ):

    if post_index != None:
        print("[ Testing post index retrieval ]")

        # Return top k results (% k)
        posts_engine = search_engine( post_index,
                model,
                metadata_keys=['docno', 'text', 'origtext','postno', 'parentno' ],
                token_pipeline=tokens ) % k

        result = batch_query( posts_engine, read_topics )
        show_result( result, [], show_hits=True )

    if math_index != None:
        print("[ Testing math index retrieval ]")

        # Return top k results (% k)
        math_engine = search_engine( math_index, model, ['docno', 'text', 'origtext','postno', 'parentno' ], token_pipeline=tokens ) % k
        show_result( query( math_engine, '_pand sqrt _pand 2' ), show_hits=True, math=True )

    print( 'Test complete.' )

In [24]:
#from index_arqmath import *
#from arqmath_topics_qrels import *
import argparse
from pyterrier.measures import *
import os, sys

# Constants
REL_THRESHOLD=2  # ARQMath convention, treat '2' in 0-3 scale as 'relevant' for binary relevance metrics
MAX_HITS=1000    # TREC / CLEF / NTCIR / FIRE / ARQMath convention (max of 1000 hits per query)


################################################################
# Input/Output 
################################################################
def load_topics( file_name ):
    ( num_topics, topics_df ) = read_topic_file( file_name )
    queries = topics_df[ 'query' ]
    qids = topics_df[ 'qid' ]
    query_df = pt.new.queries( queries, qid=qids)

    print(query_df.head()) 
    return ( num_topics, query_df )

def load_qrels_with_binary( file_name ):
    qrels_df = load_qrels( file_name )
    # !! Create thresholded qrels for map' and p'@10
   # print(qrels_df[ qrels_df['label']])
    qrels_thresholded = qrels_df[ qrels_df['label'] >= REL_THRESHOLD ]

    return ( qrels_df, qrels_thresholded )

def load_index( index_dir, lexicon, stats ):
    index_path = index_dir + "/data.properties" 
    print("Loading index defined at " + index_path + "...")
    index_ref = pt.IndexRef.of( index_path )
    index = pt.IndexFactory.of( index_ref )

    # If asked, report stats and lexicon
    view_index( index_dir, index, lexicon, stats )

    return index

def report_results( ndcg_metrics, binarized_metrics, top_k, prime ):
    # Make clear what we're using!
    prime_string = ''

    if prime:
        prime_string ="'"
    print("[[ Evaluation  ]]")
    print(" * Top-k hits evaluated: " + str(top_k ) )
    print(" * Prime metrics ('): " + str(prime) )
    print(" * !! Note that ARQMath uses prime metrics for official scores.")
    print("\nResults for nDCG" + prime_string )
    print("----------------------------------------------------------")
    print( ndcg_metrics )

    print("\nResults for binarized relevance : mAP" + prime_string + ", Precision at 10" + prime_string)
    print("----------------------------------------------------------")
    print( binarized_metrics )

    print("\ndone.")

################################################################
# Evaluation
################################################################
# Used to remove unasessed hits in search results for prime (') metrics
# Consider only up to MAX_HITS
def select_assessed_hits( qrel_df, top_k=1000, prime=True ):
    def filter_results( result_df ):
        #result_df.drop_duplicates( subset='docno' )  # esp. important for formula retrieval results
        #result_df_cut = result_df.iloc[0 : MAX_HITS ]
        result_df_cut = result_df.loc[ result_df[ 'rank' ] < top_k ]
        out_results = result_df_cut

        # Prime metrics
        # If 'prime' is true, filter by ( qid, docno ) pairs in the qrel file, so that
        # only assessed hits are included.
        if prime:
            keycols = qrel_df[ ['qid','docno'] ]
            keys = list( keycols.columns.values )
            i1 = result_df_cut.set_index(keys).index
            i2 = qrel_df.set_index(keys).index
            out_results = result_df_cut[ i1.isin(i2) ]

        return out_results

    return pt.apply.generic( filter_results )


################################################################
# Main program
################################################################
# def process_args():
    # Process command line arguments
  # parser = argparse.ArgumentParser(description="Retrieve and evaluate results for ARQMath data.")

  #   parser.add_argument('indexDir', help='Directory containing ARQMath index')
   #  parser.add_argument('xmlFile', help='ARQMath topics file (XML)')
   #  parser.add_argument('qrelFile', help='ARQMath qrels file (**needs to correspond to topic file)')
   #  parser.add_argument('-m', '--model', default="BM25", help="term weight model (default: BM25; TF_IDF + other PyTerrier built-ins are available)" )
   #  parser.add_argument('-k', '--topk', type=int, default=1000, help="select top-k hits (default: 1000)" )
  #   parser.add_argument('-nop', '--noprime', default=False, help="compute non-prime metrics (default: False)", action="store_true" )
   #  parser.add_argument('-l', '--lexicon', help='show lexicon', action="store_true" )
  #   parser.add_argument('-s', '--stats', help="show collection statistics", action="store_true" )
 #    parser.add_argument('-t', '--tokens', help="set tokenization property (none:  no stemming/stopword removal)", 
   #          default='Stopwords,PorterStemmer' )
   #  parser.add_argument('-d', '--debug', help="include debugging outputs", action="store_true" )

  #   args = parser.parse_args()
    
  #   return args

def main():
    # Process arguments
    #args = process_args()
    # Set pandas display width wider
    pd.set_option('display.max_colwidth', 150)

   # if args.tokens == 'none':
      #  args.tokens = ''

    # Set retrieval and evaluation parameters
    weight_model = "BM25"
    prime = False
    top_k = 1000

    # Do not forget, or fields are undefined ('None' in error messages)
    print('\n>>> Initializing PyTerrier...')
    if not pt.started():
        pt.init()

    print("\n>>> Starting up ")

    # Collect topics, qrels index
    print("Loading topics (queries)...")
    (num_topics, query_df ) = load_topics('Topics_V2.0.xml')
    print("    " + str(num_topics) + " topics loaded.")

    print("Loading qrels...")
    ( qrels_df, qrels_thresholded) = load_qrels_with_binary( 'qrel_official_task1' )

    print("Loading index defined at " + '/content/ARQMath_Collection-math-ptindex' + "...")
    index = load_index( '/content/ARQMath_Collection-math-ptindex' , True, True )

    # Report tokenization
    #token_pipeline = index_ref.getProperty("termpipelines")  # does not work.
    #print("Tokenization: " + token_pipeline)

    print("Generating search engine...(" + weight_model + \
            ") with tokenization spec: '" + 'Stopwords,PorterStemmer' + "')" )
    # Compiling example to make it faster (see https://pyterrier.readthedocs.io/en/latest/transformer.html)
    # * Filtering unasessed hits (w. prime_transformer) - also enforces maximum result list length.
    prime_transformer = select_assessed_hits( qrels_df, top_k, prime )
    bm25_engine = search_engine( index, weight_model, MATH_META_FIELDS, token_pipeline='Stopwords,PorterStemmer' )
    bm25_pipeline = bm25_engine >> prime_transformer

    # First pass: two runs on retrieval, one for ndcg', one for binarized metrics
    # Saves results to current directory in file <weight_model>.res.gz this prevents running
    # the retrieval pipeline in the second call.
    #
    # **See warnings about accidentally reusing results and getting incorrect results here: 
    #   https://pyterrier.readthedocs.io/en/latest/experiments.html?highlight=experiments
    #  ('overwrite' used to create new results for each run of this program for the first retrieval)
    print("Running topics...")
    ndcg_metrics = pt.Experiment(
        [ bm25_pipeline ],
        query_df,
        qrels_df,
        eval_metrics=[ "ndcg", "mrt" ],
        names=[ weight_model ],
        save_dir="./",
        save_mode="overwrite"
        )

    # This does not run the pipeline again, but does compute different metrics used the save
    # search results.
    binarized_metrics = pt.Experiment(
        [ bm25_engine ],
        query_df,
        qrels_thresholded,
        eval_metrics=[ "P_10", "map", "mrt" ],
        names=[ weight_model ],
        save_dir="./"
        )
    
    # Report results at the command line.
    report_results( ndcg_metrics, binarized_metrics, top_k, prime )

main()



Streaming output truncated to the last 5000 lines.
uuddu :    term84818 Nt=1 TF=1 maxTF=2147483647 @{0 220815924 5} TFf=1,0
uudu :    term51644 Nt=3 TF=3 maxTF=2147483647 @{0 220815931 2} TFf=3,0
uudud :    term84814 Nt=1 TF=1 maxTF=2147483647 @{0 220815949 5} TFf=1,0
uududuuddd :    term70887 Nt=1 TF=1 maxTF=2147483647 @{0 220815956 2} TFf=1,0
uuduu :    term84813 Nt=1 TF=1 maxTF=2147483647 @{0 220815962 5} TFf=1,0
uuduuddd :    term8954 Nt=1 TF=1 maxTF=2147483647 @{0 220815969 2} TFf=1,0
uuduuddudd :    term73787 Nt=1 TF=1 maxTF=2147483647 @{0 220815974 3} TFf=1,0
uuduuudddudd :    term42324 Nt=1 TF=1 maxTF=2147483647 @{0 220815980 6} TFf=1,0
uuduuudddudduuuddd :    term42325 Nt=1 TF=1 maxTF=2147483647 @{0 220815986 7} TFf=1,0
uue :    term76927 Nt=1 TF=1 maxTF=2147483647 @{0 220815993 0} TFf=1,0
uul :    term61088 Nt=2 TF=2 maxTF=2147483647 @{0 220815999 3} TFf=2,0
uuldrdruu :    term40314 Nt=1 TF=1 maxTF=2147483647 @{0 220816006 5} TFf=1,0
uurrr :    term96652 Nt=1 TF=1 maxTF=21474

In [ ]:
# Test post retrieval
k = 3
test_retrieval( k, post_index, None, 'BM25', 'Stopwords,PorterStemmer', debug=False )

In [ ]:
# Test math retrieval
k = 3
test_retrieval( k, None, math_index, 'BM25', 'Stopwords,PorterStemmer', debug=False )

In [26]:
import os
from subprocess import check_output
import argparse

def calculated_map(res_directory, trec_eval_tool, qre_file_path):
    result = {}
    for file in os.listdir(res_directory):
        output = check_output([trec_eval_tool, qre_file_path, res_directory+file, "-l2", "-m", "map"])
        output = output.decode('utf-8')
        score = output.split("\t")[2].strip()
        submission = file.split(".")[0].split("prime_")[1]
        result[submission] = score
    return result

In [37]:
calculated_map('/content/ugub', 'trec_eval','/content/qrel_official_task1')


FileNotFoundError: ignored

In [ ]:
# #load relevance judgements into training & validation sets
# from pv211_utils.arqmath.loader import load_judgements

# smaller_train_judgements = load_judgements(smaller_train_queries, answers, 'train', year=2020)
# smaller_validation_judgements = load_judgements(smaller_validation_queries, answers, 'validation', year=2020)

# train_judgements = smaller_train_judgements | smaller_validation_judgements
# validation_judgements = load_judgements(validation_queries, answers, 'test', year=2020)

# bigger_train_judgements = train_judgements | validation_judgements

# pivot = int(len(train_judgements) * 0.8)
# smaller_train_judgements = set(sorted(train_judgements)[:pivot])
# smaller_validation_judgements = set(sorted(train_judgements)[pivot:])

# print(type(train_judgements))

# print(train_judgements)
# print('Number of Judgments:', len(bigger_train_judgements))

In [ ]:
# for bert
# !git clone --recursive https://github.com/castorini/pygaggle.git
# !pip install -r pygaggle/requirements.txt 
# !pip install -e /content/pygaggle
# !pip install transformers==4.6.1

# ## Installing the faiss library which's integral to pygaggle
# !apt install libomp-dev
# !pip install faiss
# !pip install faiss-gpu



In [ ]:
# #evaluation
# from pv211_utils.arqmath.leaderboard import ArqmathLeaderboard
# from pv211_utils.arqmath.eval import ArqmathEvaluation

# submit_result = False
# author_name = 'Surname, Name'

# print('Initializing your system ...')
# system = IRSystem()

# test_queries = train_queries
# test_judgements = train_judgements

# #Don't uncomment these lines until you are ready to do the final runs for submission!
# #test_queries = load_queries(text_format, Query, year=2021)
# #test_judgements = load_judgements(test_queries, answers, year=2021)

# evaluation = ArqmathEvaluation(system, test_judgements, ArqmathLeaderboard(), author_name, num_workers=1)

# evaluation.evaluate(tqdm(test_queries.values(), desc='Querying the system'), submit_result)